In [1]:
import os

In [2]:
%pwd

'c:\\Users\\SACHIN\\PycharmProjects\\Obesity-Risk-\\notebook'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\SACHIN\\PycharmProjects\\Obesity-Risk-'

In [46]:
from pathlib import Path
import os
from dataclasses import dataclass


@dataclass(frozen=True)
class ModelTrainerConfig:
    root : Path
    model : Path
    grow_policy:  str
    n_estimators: int
    learning_rate: float
    gamma :   float
    subsample: float
    colsample_bytree: float
    max_depth: int
    min_child_weight: int
    reg_lambda: float
    reg_alpha: float
    


@dataclass(frozen= True)
class DataTransformationConfig:
    root : Path
    data: Path
    train_set : Path
    test_set: Path
    preprocessing_obj: Path

In [47]:
from src.ObesityRisk.constants import *
from src.ObesityRisk.utils.common import read_yaml, create_directories

In [49]:
class ConfigurationsManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_file])

    def get_transformation_config(self)-> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root])
        create_directories([config.train_set])
        create_directories([config.test_set])
        #create_directories([config.preprocessing_obj])

        data_transformation_config = DataTransformationConfig(
            root = config.root,
            data = config.data,
            train_set = config.train_set,
            test_set = config.test_set,
            preprocessing_obj = config.preprocessing_obj
        )

        return data_transformation_config
        

    
    def get_model_trainer(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params

        create_directories([config.root])

        model_trainer_config = ModelTrainerConfig(
            root = Path(config.root),
            model = Path(config.model),
            grow_policy=  params.grow_policy,
            n_estimators= params.n_estimators,
            learning_rate= params.learning_rate,
            gamma =   params.gamma,
            subsample= params.subsample,
            colsample_bytree= params.colsample_bytree,
            max_depth= params.max_depth,
            min_child_weight= params.min_child_weight,
            reg_lambda= params.reg_lambda,
            reg_alpha= params.reg_alpha,
            
        )

        return model_trainer_config


In [50]:
import os

import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.metrics import accuracy_score

from ObesityRisk.utils.common import *
from ObesityRisk import logger
from ObesityRisk.constants import *
from ObesityRisk.components.data_transformation import PrepareTransformation
import warnings
warnings.filterwarnings('ignore')

In [63]:
class PrepareModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def initiate_model_trainer(self,train_data, test_data):
        logger.info("Initializing model trainer")

        logger.info("Slpitting data into Train and Test")
        X_train, y_train, X_test, y_test = (
                train_data[:, :-1],
                train_data[:, -1],
                test_data[:, :-1],
                test_data[:, -1]
                )

        print(train_data[:,-1])
        print(test_data[:,-1])
        print(y_train)
        print(y_test)

        

        logger.info('fitting train data into model') 

        xcls = XGBClassifier(self.config.grow_policy, self.config.n_estimators, self.config.learning_rate, self.config.gamma,
                            self.config.subsample, self.config.colsample_bytree, self.config.max_depth, 
                            self.config.min_child_weight, self.config.reg_lambda, self.config.reg_alpha)
        xcls.fit(X_train, y_train)
        #rcv = RandomizedSearchCV( estimator= xcls, param_distributions=config.params, n_iter=10,cv=5, n_jobs=1,scoring='accuracy')
        #rcv.fit(X_train, y_train)
        #pred = rcv.predict(X_test)
        #ac = accuracy_score(y_test, pred)
        #print(ac)

        save_pickle(os.path.join(self.config.model, "model.pkl"),xcls)

        return xcls
        



In [64]:
try:
    config = ConfigurationsManager()
    base_model_config = config.get_transformation_config()
    data_transformation = PrepareTransformation(config=base_model_config)
    train_data, test_data = data_transformation.data_split()
    train_arr,test_arr = data_transformation.initiate_data_transformation(train_data, test_data)

    get_model_trainer = config.get_model_trainer()
    model_trainer = PrepareModelTrainer(config=get_model_trainer)
    model_trainer.initiate_model_trainer(train_arr,test_arr)
except Exception as e:
    raise e

[2024-03-07 23:22:24,990: INFO: common: yaml file Config\Config.yaml loaded successfully]
[2024-03-07 23:22:25,001: INFO: common: yaml file params.yaml loaded successfully]
[2024-03-07 23:22:25,006: INFO: common: creating directory at artifacts]
[2024-03-07 23:22:25,009: INFO: common: creating directory at artifacts/data_transformation]
[2024-03-07 23:22:25,012: INFO: common: creating directory at artifacts/data_transformation/train_set]
[2024-03-07 23:22:25,015: INFO: common: creating directory at artifacts/data_transformation/test_set]


[2024-03-07 23:22:25,696: INFO: data_transformation:           id  Gender        Age    Height      Weight  \
9958    9958    Male  17.000000  1.770000   97.000000   
7841    7841    Male  22.667596  1.753389   54.877111   
9293    9293    Male  21.838323  1.819867  122.024954   
15209  15209  Female  41.000000  1.580000   80.000000   
16515  16515    Male  23.000000  1.800000   95.000000   
...      ...     ...        ...       ...         ...   
11284  11284  Female  18.000000  1.722461   80.442775   
11964  11964    Male  22.000000  1.750000   95.000000   
5390    5390    Male  21.000000  1.620000   68.000000   
860      860  Female  26.000000  1.650125  111.939671   
15795  15795    Male  37.997912  1.774330  107.998815   

      family_history_with_overweight FAVC      FCVC       NCP       CAEC  \
9958                             yes  yes  3.000000  3.000000     Always   
7841                             yes  yes  2.000000  4.000000  Sometimes   
9293                             y